In [1]:
pip install beautifulsoup4 pandas lxml

   ---------------------------------------- 0.0/4.0 MB ? eta -:--:--
   -- ------------------------------------- 0.3/4.0 MB ? eta -:--:--
   ----- ---------------------------------- 0.5/4.0 MB 1.6 MB/s eta 0:00:03
   ---------- ----------------------------- 1.0/4.0 MB 1.9 MB/s eta 0:00:02
   --------------- ------------------------ 1.6/4.0 MB 2.1 MB/s eta 0:00:02
   -------------------- ------------------- 2.1/4.0 MB 2.2 MB/s eta 0:00:01
   -------------------------- ------------- 2.6/4.0 MB 2.2 MB/s eta 0:00:01
   ------------------------------- -------- 3.1/4.0 MB 2.4 MB/s eta 0:00:01
   ---------------------------------------  3.9/4.0 MB 2.5 MB/s eta 0:00:01
   ---------------------------------------- 4.0/4.0 MB 2.5 MB/s  0:00:01
Note: you may need to restart the kernel to use updated packages.
